In [ ]:
pwd

In [ ]:
!wget https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip

In [ ]:
!unzip -qq cats_and_dogs_filtered.zip

In [ ]:
ls

In [ ]:
import os
from torch.utils.data import Dataset
import torchvision.transforms as transforms 
from PIL import Image


In [ ]:
import torch
print(torch.__version__)


In [ ]:
is_cuda = False
if torch.cuda.is_available():
    is_cuda = True

파이토치 데이터셋 클래스 

In [ ]:
import os
from torch.utils.data import Dataset
import torchvision.transforms as transforms
from PIL import Image

class PyTorchCustomDataset(Dataset):
    def __init__(self
                 , root_dir = "/content/cats_and_dogs_filtered/train"
                 , transform = None):
        self.image_abs_path = root_dir
        self.transform = transform
        self.label_list = os.listdir(self.image_abs_path)
        self.label_list.sort()
        self.x_list = []
        self.y_list = []
        for label_index, label_str in enumerate(self.label_list):
            img_path = os.path.join(self.image_abs_path, label_str)
            img_list = os.listdir(img_path)
            for img in img_list:
                self.x_list.append(os.path.join(img_path, img))
                self.y_list.append(label_index)
        pass

    def __len__(self):
        return len(self.x_list)

    def __getitem__(self, idx):
        image = Image.open(self.x_list[idx])
        if image.mode is not "RGB":
            image = image.convert('RGB')
        if self.transform is not None:
            image = self.transform(image)
        return image, self.y_list[idx]

    def __save_label_map__(self, dst_text_path = "label_map.txt"):
        label_list = self.label_list
        f = open(dst_text_path, 'w')
        for i in range(len(label_list)):
            f.write(label_list[i]+'\n')
        f.close()
        pass

    def __num_classes__(self):
        return len(self.label_list)

Model

In [ ]:
import torch
from torchvision import models
import torch.nn as nn
import torch.nn.functional as F

class MODEL(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.network = models.resnet18(pretrained=True)
        self.classifier = nn.Sequential(
            nn.Dropout()
            , nn.Linear(1000, num_classes)
            , nn.Sigmoid()
        )
    def forward(self, x):
        x = self.network(x)
        return self.classifier(x)

main 함수

In [ ]:
import torch
import torch.optim as optim

train_losses , train_accuracy = [],[]
val_losses , val_accuracy = [],[] 

def main():
    USE_CUDA = torch.cuda.is_available()
    DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
    img_width, img_height = 224, 224
    EPOCHS     = 10
    BATCH_SIZE = 16
    transform_train = transforms.Compose([
                transforms.Resize(size=(img_width, img_height))
                , transforms.RandomRotation(degrees=15)
                , transforms.ToTensor()
                ])
    transform_test = transforms.Compose([
                transforms.Resize(size=(img_width, img_height))
                , transforms.ToTensor()
                ])
    TrainDataset = PyTorchCustomDataset
    TestDataset = PyTorchCustomDataset
    train_data = TrainDataset(root_dir = "/content/cats_and_dogs_filtered/train"
                    , transform = transform_train)
    test_data = TestDataset(root_dir = "/content/cats_and_dogs_filtered/validation"
                    , transform = transform_test)
    train_loader = torch.utils.data.DataLoader(
        train_data
        , batch_size=BATCH_SIZE
        , shuffle=True
    )
    test_loader = torch.utils.data.DataLoader(
        test_data
        , batch_size=BATCH_SIZE
        , shuffle=True
    )
    train_data.__save_label_map__()
    num_classes = train_data.__num_classes__()
    model = MODEL(num_classes).to(DEVICE)
    model_str = "PyTorch_Classification_Model"
    model_str += ".pt" 
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)
    acc = 0.0
   

    for epoch in range(1, EPOCHS + 1):
        model.train()
        tr_loss = 0.0
        tr_correct = 0.0       
        for data, target in (train_loader):
            data, target = data.to(DEVICE), target.to(DEVICE)
            optimizer.zero_grad()
            output = model(data)
            loss = F.cross_entropy(output, target)
            tr_loss += F.nll_loss(output,target,reduction='sum').item()
            pred = output.data.max(dim=1,keepdim=True)[1]
            tr_correct += pred.eq(target.view_as(pred)).sum().item()          
            loss.backward()
            optimizer.step()
        scheduler.step()
        tr_ep_loss = tr_loss/len(train_loader.dataset)
        tr_ep_accuracy = 100. * tr_correct/len(train_loader.dataset)

        model.eval()
        te_loss = 0
        te_correct = 0
        with torch.no_grad():
            for data, target in (test_loader):
                data, target = data.to(DEVICE), target.to(DEVICE)
                output = model(data)
                loss = F.cross_entropy(output, target)
                te_loss += F.cross_entropy(output, target, reduction='sum').item()
                pred = output.max(1, keepdim=True)[1]
                te_correct += pred.eq(target.view_as(pred)).sum().item()
        te_ep_loss = te_loss / len(test_loader.dataset)
        te_ep_accuracy = 100. * te_correct / len(test_loader.dataset)
        print('[{}] Train Loss: {:.4f}, Train Accuracy: {:.2f}% Test Loss: {:.4f}, Test Accuracy: {:.2f}%'.format(
                epoch, tr_ep_loss, tr_ep_accuracy, te_ep_loss, te_ep_accuracy))
    
        if acc < te_ep_accuracy:
            acc = te_ep_accuracy
            torch.save(model.state_dict(), model_str)
            print("model saved!")

        train_losses.append(tr_ep_loss)
        train_accuracy.append(tr_ep_accuracy)
        val_losses.append(te_ep_loss)
        val_accuracy.append(te_ep_accuracy)



In [ ]:
main()

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(1,len(train_losses)+1),train_losses,'bo',label = 'training loss')
plt.plot(range(1,len(val_losses)+1),val_losses,'r',label = 'validation loss')
plt.legend()

In [ ]:
plt.plot(range(1,len(train_accuracy)+1),train_accuracy,'bo',label = 'train accuracy')
plt.plot(range(1,len(val_accuracy)+1),val_accuracy,'r',label = 'val accuracy')
plt.legend()

가중치파일(PyTorch_Classification_Model.pt) 
과 “pytorch_network_model.py” 파일, “label_map.txt.”파일을 colab에 업로드합니다.

